1. Load matches

In [1]:
from pathlib import Path
import yaml

matches = yaml.safe_load(Path('/home/ilianolhin/git/nerpa2/training/training/matches_inspection_results/approved_matches.yaml').read_text())

2. Load BGC variants

In [2]:
from collections import defaultdict
nerpa_results_dir = Path('/home/ilianolhin/git/nerpa2/training/training/nerpa_results/currently_inspecting/individual')
bgc_variants = defaultdict(list)
for bgc_dir in nerpa_results_dir.iterdir():
    bgc_id = bgc_dir.name
    try:
        yaml_files = [f for f in (bgc_dir / 'BGC_variants').iterdir() if f.name.endswith('.yaml')]
    except FileNotFoundError:
        print(f'No BGC variants for {bgc_id}')
        continue
    for yaml_file in yaml_files:
        bgc_variants[bgc_id].extend(yaml.safe_load(yaml_file.read_text()))
        

No BGC variants for BGC0002100


3. Extract distinct alignment steps

In [3]:
def step_key(match, step) -> tuple:
    return (match['Genome'], 
            step['Gene'], 
            step['A-domain_idx'], 
            step['Modifying_domains'],
            step['NRP_chirality'],
            step['NRP_modifications'],
            step['NRP_residue'],
            step['Alignment_step'])

all_steps_dict = {}
for match in matches:
    match_steps = [alignment_step for alignment in match['Alignments']
                   for alignment_step in alignment]
    for step in match_steps:
        key = step_key(match, step)
        all_steps_dict[key] = step
            
all_steps = list(all_steps_dict.values())

3. Count mismatches

In [4]:
from collections import Counter
mismatches = Counter()
used_steps = set()
for match in matches:
    bgc_id = match['Genome']
    bgc_variant = next(variant for variant in bgc_variants[bgc_id] 
                       if variant['variant_idx'] == match['BGC_variant_idx'])
    for alignment in match['Alignments']:
        for step in alignment:
            if step_key(match, step) in used_steps:
                continue
            used_steps.add(step_key(match, step))
            if step['Alignment_step'] != 'MATCH':
                continue
            top_scoring_residues = step['Top_scoring_residues'].split(',')
            if step['NRP_residue'] in top_scoring_residues:
                continue
            specificity_predictions = next(module['residue_score'] 
                                           for module in bgc_variant['tentative_assembly_line']
                                           if module['gene_id'] == step['Gene'] and module['module_idx'] == step['A-domain_idx'])
            top_score = max(specificity_predictions.values())
            if top_score == 0.0:
                for res in top_scoring_residues:
                    mismatches[(res, step['NRP_residue'])] += 1

with open('frequently_mismatches_pairs.tsv', 'w') as f:
    for (res1, res2), count in mismatches.most_common():
        f.write(f'{res1}\t{res2}\t{count}\n')

4. Score calibration

In [5]:
score_correctness = []
used_steps = set()
for match in matches:
    bgc_id = match['Genome']
    bgc_variant = next(variant for variant in bgc_variants[bgc_id]
                       if variant['variant_idx'] == match['BGC_variant_idx'])
    for alignment in match['Alignments']:
        for step in alignment:
            if step_key(match, step) in used_steps:
                continue
            used_steps.add(step_key(match, step))
            if step['Alignment_step'] != 'MATCH':
                continue
            specificity_predictions = next(module['residue_score']
                                           for module in bgc_variant['tentative_assembly_line']
                                           if module['gene_id'] == step['Gene'] and module['module_idx'] == step['A-domain_idx'])
            for res, score in specificity_predictions.items():
                score_correctness.append((score, res == step['NRP_residue']))
